# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [11]:
import pickle
import re

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

## Подготовка

Возможно, мы нее впервые обрабатываем эти данные и у нас есть заначка...

In [3]:
try:
    corpus = pd.read_csv("./toxic_comments.csv")
except FileNotFoundError:
    corpus = pd.read_csv("/datasets/toxic_comments.csv")

In [4]:
corpus.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные получены. Это почть 160к комментариев с признаком токсичности. Тексты на английском, хотя попадаются и вкрапления, в том числе и на русском. 

In [6]:
corpus[corpus["text"].str.contains("и")].head()

,text,toxic
126,"Ahh, Hello Witzeman \n\n203.92.84.161 \nSymbo...",0
228,http://www.users.bigpond.com/MONTDALE/page8.ht...,0
2652,I understand your difficulty. I am able to rea...,0
3156,"""\n\nMaybe rather stop campaigning? MY was the...",0
7648,"""\nComment Hi There, In spite all my effort, I...",0


In [7]:
corpus.loc[2652, "text"]

'I understand your difficulty. I am able to read some other language Wikis, but I can not write an article for them. On a different page,  explained that a stamp article about Azerbaijan was deleted about a year ago. I do not know what happened there exactly. Do you know that there is a Wiki site where people can request translations Wikipedia articles from other languages into English? If the article ru:Почтовые марки и история почты Азербайджана is exactly what the English Wiki article is supposed to contain, perhaps you could ask there for someone to translate it. There is a small form to fill out there. I can even do that for you, if you tell me the article is exactly what is needed here. I can not promise that anyone will translate it, but we can ask. Thank you again for your willingness to help.'

Стало быть для дальнейшей работы нам потребуется корпус английских стоп-слов 

### Лемматизация

Будем сохранять промежуточные результаты на диск, чтобы потом не было мучительно больно...

In [8]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /home/sam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для дальнейшей работы тексты нужно лемматизировать и токенизировать. В ряде случаев для правильной лемматизации лемматезатору требуется уточнение, какой частью речи является слова (POS). Эту информацию также можно получить средствами nltk 

In [9]:
def pos_tag(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV,
    }
    # если слово не является существительным, прилагательным, гаголом или наречием, назначаем его существительным
    return tag_dict.get(tag, wordnet.NOUN)

In [20]:
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to /home/sam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /home/sam/nltk_data...


True

In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
%%time
corpus["lemm_text"] = corpus["text"].apply(
    lambda x: " ".join(
        [lemmatizer.lemmatize(w, pos_tag(w)) for w in nltk.word_tokenize(x)]
    )
)

CPU times: user 13min 45s, sys: 45.2 s, total: 14min 30s
Wall time: 14min 30s


In [25]:
corpus["lemm_text"] = corpus["lemm_text"].apply(
    lambda x: (" ".join(re.sub(r"[^a-zA-Z ]", " ", x).split()).lower())
)

In [26]:
corpus.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


In [27]:
with open("lemmatized.pickle", "wb") as f:
    pickle.dump(corpus, f)

Создадим для начала простой мешок слов и TF-IDF

In [28]:
cvect = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

In [29]:
tfvect = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3))

Разобъем наш корпус текстов на тренировочную и тестовую выборки и обучим векторизатор на тренировочной части

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    corpus["lemm_text"], corpus["toxic"], test_size=0.2, random_state=42
)

Было бы интересно попробовать снизить размерность матриц с помощью алгоритма SVD, благо они весьма разреженные, это могло бы благоприятно отразиться на стокорости вычислений. Но, увы, мой компьютер не осилил.

### Вывод

Данные скачаны и подготовлены к обучению. Были проведены:

- лемматизация
- построение "мешка слов"
- векторизация по алгоритму TF-IDF
- данные поделены на обучающую и тестовую выборки (4:1)

## Обучение

В качестве классификатора будем использовать логистическую регрессию: более сложные модели на пространстве признаков такой размерности мгновенно переобучатся, да и работать будут, скорее всего, очень медленно 

Начнём с "мешка слов"

In [31]:
grid_params = [
    {
        "clf__penalty": ["l2"],
        "clf__C": [5e4, 1e5, 2e5],  # np.logspace(1, 10, 5),
    }
]
pipe = Pipeline(
    [
        ("vect", CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))),
        ("clf", LogisticRegression(n_jobs=-1, random_state=42, max_iter=50)),
    ]
)

cv_bow = GridSearchCV(estimator=pipe, param_grid=grid_params, scoring="f1", cv=3)

In [32]:
%%time

try:
    with open("cv_bow.pickle", "rb") as f:
        cv_bow = pickle.load(f)
except FileNotFoundError:
    cv_bow.fit(X_train, y_train)
    with open("cv_bow.pickle", "wb") as f:
        pickle.dump(cv_bow, f)

/home/sam/.local/share/virtualenvs/00_Praktikum-W4BAQCOP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sam/.local/share/virtualenvs/00_Praktikum-W4BAQCOP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    h

CPU times: user 4min 1s, sys: 7 s, total: 4min 8s
Wall time: 9min 23s


In [33]:
cv_bow.best_params_

{'clf__C': 200000.0, 'clf__penalty': 'l2'}

In [34]:
print(f"F1 on train: {f1_score(y_train, cv_bow.best_estimator_.predict(X_train))}")
print(f"F1 on test: {f1_score(y_test,  cv_bow.best_estimator_.predict(X_test))}")

F1 on train: 0.9912781722753936
F1 on test: 0.7849908683380376


Собственно, желаемая метрика качества достигнута

Теперь TF-IDF

In [35]:
grid_params = [
    {
        "clf__penalty": ["l2"],
        "clf__C": np.logspace(-3, 5, 5),
    }
]
pipe = Pipeline(
    [
        ("vect", TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3))),
        ("clf", LogisticRegression(n_jobs=-1, random_state=42, max_iter=50)),
    ]
)

cv_tf_idf = GridSearchCV(estimator=pipe, param_grid=grid_params, scoring="f1", cv=3)

In [36]:
%%time

try:
    with open("cv_tf_idf.pickle", "rb") as f:
        cv_tf_idf = pickle.load(f)
except FileNotFoundError:
    cv_tf_idf.fit(X_train, y_train)
    with open("cv_tf_idf.pickle", "wb") as f:
        pickle.dump(cv_bow, f)

/home/sam/.local/share/virtualenvs/00_Praktikum-W4BAQCOP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sam/.local/share/virtualenvs/00_Praktikum-W4BAQCOP/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    h

CPU times: user 6min 50s, sys: 10.6 s, total: 7min
Wall time: 13min 41s


In [37]:
cv_tf_idf.best_params_

{'clf__C': 1000.0, 'clf__penalty': 'l2'}

In [38]:
print(f"F1 on train: {f1_score(y_train, cv_tf_idf.best_estimator_.predict(X_train))}")
print(f"F1 on test: {f1_score(y_test,  cv_tf_idf.best_estimator_.predict(X_test))}")

F1 on train: 0.992459218220991
F1 on test: 0.7916075090708314


## Выводы

Для анализа текстов использовалась векторизация с помощью мешка слов и TF-IDF. Оба метода показали хорошие результаты и оба достигли желаемой метрики качества ($F_1>0.75$):
- "Мешок слов": $F_1=0.78$    
- TF-IDF: $F_1=0.79$

К сожалению, попробовать BERT на имеющемся датасете не удалось: после подготовки токенов, структуды данных в памяти занимали более 13Гб (из 16 имеющихся), что сделало невозможной дальнейшую работу.

<div class="alert alert-block alert-success">
<b>Успех:</b> Приятно видеть вывод в конце проекта!
</div>